<a href="https://colab.research.google.com/github/yananma/5_programs_per_day/blob/master/04205.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 4.2 模型参数的访问、初始化和共享

In [2]:
import torch 
from torch import nn 
from torch.nn import init 

net = nn.Sequential(nn.Linear(4, 3), nn.ReLU(), nn.Linear(3, 1))
net 

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)

In [3]:
X = torch.rand(2, 4)
net(X)

tensor([[0.2371],
        [0.2078]], grad_fn=<AddmmBackward>)

In [4]:
Y = net(X).sum()
Y

tensor(0.4448, grad_fn=<SumBackward0>)

### 4.2.1 访问模型参数

In [5]:
print(type(net.named_parameters()))

<class 'generator'>


In [6]:
for name, param in net.named_parameters():
    print(name, param.size())

0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


In [7]:
for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


In [8]:
class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.weight1 = nn.Parameter(torch.rand(20, 20))
        self.weight2 = torch.rand(20, 20)

    def forward(self, x):
        pass 


n = MyModel()
for name, param in n.named_parameters():
    print(name)

weight1


In [9]:
weight_0 = list(net[0].parameters())[0]
weight_0.data

tensor([[ 0.3120,  0.3832, -0.2590, -0.2786],
        [-0.3275, -0.3611, -0.1754, -0.3223],
        [-0.2355,  0.1803, -0.0546, -0.4812]])

In [10]:
print(weight_0.grad)

None


In [11]:
Y.backward()
weight_0.grad

tensor([[0.2141, 0.1815, 0.1700, 0.1521],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000]])

### 4.2.2 初始化模型参数

In [12]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0, std=0.01)
        print(name, param.data)

0.weight tensor([[ 0.0114,  0.0106,  0.0060,  0.0084],
        [ 0.0190, -0.0069,  0.0128,  0.0150],
        [-0.0085, -0.0034, -0.0051, -0.0029]])
2.weight tensor([[ 0.0094, -0.0043, -0.0024]])


In [13]:
for name, param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param, val=0)
        print(name, param.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


### 4.2.3 自定义初始化方法

In [0]:
def normal_(tensor, mean=0, std=1):
    with torch.no_grad():
        return tensor.normal_(mean, std)

In [15]:
def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10, 10)
        tensor *= (tensor.abs() >= 5).float()


for name, param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name, param.data)

0.weight tensor([[ 6.3236,  6.5301,  0.0000, -0.0000],
        [-5.0343,  0.0000,  6.4923, -0.0000],
        [ 0.0000, -9.0378,  9.8344,  0.0000]])
2.weight tensor([[-5.5816,  5.1283,  8.0439]])


In [16]:
for name, param in net.named_parameters():
    if 'bias' in name:
        param.data += 1 
        print(name, param.data)

0.bias tensor([1., 1., 1.])
2.bias tensor([1.])


### 4.2.4 共享参数模型

In [17]:
linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear, linear)
net 

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)

In [19]:
for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)

0.weight tensor([[3.]])


In [20]:
id(net[0]) == id(net[1])

True

In [21]:
id(net[0].weight) == id(net[1].weight)

True

In [22]:
x = torch.ones(1, 1)
y = net(x).sum()
y

tensor(9., grad_fn=<SumBackward0>)

In [23]:
y.backward()
net[0].weight.grad

tensor([[6.]])